<a href="https://colab.research.google.com/github/PeterStoreshaw/PeterStoreshaw.github.io/blob/master/Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Question 3, Fake vs Real Images

To Begin with we will use the code from fast AI
'bird or not'

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai

Install Fastbook

In [ ]:
!pip install fastbook

If using google colab we can just grab those photos already downloaded.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/archive.zip'

path = Path('train') 

If the previous step was run and successful SKIP THIS STEP.

## Step 2: Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

After this step I went into the folders and manualy deleted all of the phots that do not fit. There were so so many lamps. The process is detailed in my blog.

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Create the data block.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

Run the learner. Here we can change the aritecture to see if there are any differnces. As well as the fine tuning. When comparing the confusion matrix we can improve the program.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)

## Step 3: Check Accuracy

Create and plot a confusion matrix


The confusion matrix will help identify the accuracy. 
It works by finding the true negative, true positive, false negative and false positive for each selection. 
So, if we predicted cat, and it was a cat that is a true positive. 
The matrix this code will create just looks at true positives and incorrect choices. If the majority of diagonal line from left to right is all 100 percent we know that the system is perfectly accurate. 


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Show the worst 8 photos for confusion.
 This shows us the 8 most troublesome photos that the sytem is stugling to correctly identify.

In [ ]:
interp.plot_top_losses(8, nrows=1)


## Step 4: Prediction

In [ ]:
guess,_,probs = learn.predict(PILImage.create('images.jpg'))
print(f"This is a: {guess}.")
